# Sigma

The `sigma` project defines a generic, `YAML`-based pattern matching syntax (similar to `yara`) that allows hunters to define their logic in a database-agnostic way.

From `rules/windows/process_creation/win_local_system_owner_account_discovery.yml`

    title: Local Accounts Discovery
    id: 502b42de-4306-40b4-9596-6f590c81f073
    status: experimental
    description: Local accounts, System Owner/User discovery using operating systems utilities
    author: Timur Zinniatullin, Daniil Yugoslavskiy, oscd.community
    date: 2019/10/21
    modified: 2019/11/04
    references:
        - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1033/T1033.yaml
    logsource:
        category: process_creation
        product: windows
    detection:
        selection_1:
          - Image|endswith: '\whoami.exe'
          - Image|endswith: '\wmic.exe'
            CommandLine|contains|all:
                - 'useraccount'
                - 'get'
          - Image|endswith: 
                - '\quser.exe'
                - '\qwinsta.exe'
          - Image|endswith: '\cmdkey.exe'
            CommandLine|contains: '/list'
          - Image|endswith: '\cmd.exe'
            CommandLine|contains|all: 
                - '/c'
                - 'dir '
                - '\Users\'
        filter_1:
            CommandLine|contains:
                - ' rmdir '       # don't match on 'dir'   "C:\Windows\System32\cmd.exe" /q /c rmdir /s /q "C:\Users\XX\AppData\Local\Microsoft\OneDrive\19.232.1124.0005"
        selection_2:
            Image|endswith:
                - '\net.exe'
                - '\net1.exe'
            CommandLine|contains: 'user'
        filter_2:
            CommandLine|contains:
                - '/domain'       # local account discovery only
                - '/add'          # discovery only
                - '/delete'       # discovery only
                - '/active'       # discovery only
                - '/expires'      # discovery only
                - '/passwordreq'  # discovery only
                - '/scriptpath'   # discovery only
                - '/times'        # discovery only
                - '/workstations' # discovery only
        condition: (selection_1 and not filter_1) or ( selection_2 and not filter_2)
    fields:
        - Image
        - CommandLine
        - User
        - LogonGuid
        - Hashes
        - ParentProcessGuid
        - ParentCommandLine
    falsepositives:
         - Legitimate administrator or user enumerates local users for legitimate reason
    level: low
    tags:
        - attack.discovery
        - attack.t1033
        - attack.t1087

## Basic Usage

`sigma` also provides a toolchain to convert these rules into search queries that are consumable by a number of different backend databases and data models. The tool `sigmac` handles the conversion from the `sigma` rule syntax into a query that your database can understand. The `--help` text can be a little intimidating...

In [1]:
!python ../tools/sigmac --help

usage: sigmac [-h] [--recurse] [--filter FILTER]
              [--target {es-qs,es-dsl,ala,qradar,grep,kibana,splunk,logpoint,sumologic,netwitness,arcsight-esm,elastalert,carbonblack,limacharlie,ala-rule,xpack-watcher,powershell,qualys,splunkxml,sql,fieldlist,arcsight,logiq,es-rule,mdatp,graylog,elastalert-dsl}]
              [--lists] [--config CONFIG] [--output OUTPUT]
              [--backend-option BACKEND_OPTION]
              [--backend-config BACKEND_CONFIG] [--defer-abort]
              [--ignore-backend-errors] [--verbose] [--debug]
              [inputs [inputs ...]]

Convert Sigma rules into SIEM signatures.

positional arguments:
  inputs                Sigma input files ('-' for stdin)

optional arguments:
  -h, --help            show this help message and exit
  --recurse, -r         Use directory as input (recurse into subdirectories is
                        not implemented yet)
  --filter FILTER, -f FILTER
                        Define comma-separated filters that mu

But really there are only a few core concepts that we need to introduce. To start, we can use the `--list` flag to identify the `target` and `config` options that we need to pass on the commandline:

In [3]:
!python ../tools/sigmac --list

Backends:
            ala : Converts Sigma rule into Azure Log Analytics Queries.
       ala-rule : Converts Sigma rule into Azure Log Analytics Rule.
       arcsight : Converts Sigma rule into ArcSight saved search. Contributed by SOC Prime. https://socprime.com
   arcsight-esm : Converts Sigma rule into ArcSight ESM saved search. Contributed by SOC Prime. https://socprime.com
    carbonblack : Converts Sigma rule into CarbonBlack query string. Only searches, no aggregations.
     elastalert : Elastalert backend
 elastalert-dsl : Elastalert backend
         es-dsl : ElasticSearch DSL backend
          es-qs : Converts Sigma rule into Elasticsearch query string. Only searches, no aggregations.
        es-rule : None
      fieldlist : List all fieldnames from given Sigma rules for creation of a field mapping configuration.
        graylog : Converts Sigma rule into Graylog query string. Only searches, no aggregations.
           grep : Generates Perl compatible regular expressions and p

The `target` (or `backend`) for the `sigma` conversion defines the syntax that the resulting query will have. The `config` defines the data model that converts field names from one schema to another. Since we are interested in converting `sigma` rules into Carbon Black queries, we will add `--target carbonblack` and `--config carbon-black` to our commandline:

In [14]:
!python ../tools/sigmac --target carbonblack --config carbon-black ../rules/windows/process_creation/win_local_system_owner_account_discovery.yml

(((process_name:whoami.exe OR (process_name:wmic.exe AND cmdline:useraccount* AND cmdline:get*) OR process_name:quser.exe OR process_name:qwinsta.exe OR (process_name:cmdkey.exe AND cmdline:/list*) OR (process_name:cmd.exe AND cmdline:/c* AND cmdline:dir\ * AND cmdline:Users*)) AND ( -(cmdline:rmdir\ *))) OR ((process_name:net.exe OR process_name:net1.exe AND cmdline:user*) AND ( -(cmdline:/domain* OR cmdline:/add* OR cmdline:/delete* OR cmdline:/active* OR cmdline:/expires* OR cmdline:/passwordreq* OR cmdline:/scriptpath* OR cmdline:/times* OR cmdline:/workstations*))))


If we indexed Windows Security/4688 events using `WinLogBeat` and wanted to find the same activity in `elasticsearch`, we could pass a different `target` syntax to convert the same rule to different query syntaxes. For example, here is the `lucene` query string version of the same `sigma` rule:

In [22]:
!python ../tools/sigmac --target es-qs --config winlogbeat ../rules/windows/process_creation/win_local_system_owner_account_discovery.yml

(((winlog.event_data.Image.keyword:*\\whoami.exe OR (winlog.event_data.Image.keyword:*\\wmic.exe AND winlog.event_data.CommandLine.keyword:*useraccount* AND winlog.event_data.CommandLine.keyword:*get*) OR winlog.event_data.Image.keyword:(*\\quser.exe OR *\\qwinsta.exe) OR (winlog.event_data.Image.keyword:*\\cmdkey.exe AND winlog.event_data.CommandLine.keyword:*\/list*) OR (winlog.event_data.Image.keyword:*\\cmd.exe AND winlog.event_data.CommandLine.keyword:*\/c* AND winlog.event_data.CommandLine.keyword:*dir\ * AND winlog.event_data.CommandLine.keyword:*\\Users\*)) AND (NOT (winlog.event_data.CommandLine.keyword:(*\ rmdir\ *)))) OR ((winlog.event_data.Image.keyword:(*\\net.exe OR *\\net1.exe) AND winlog.event_data.CommandLine.keyword:*user*) AND (NOT (winlog.event_data.CommandLine.keyword:(*\/domain* OR *\/add* OR *\/delete* OR *\/active* OR *\/expires* OR *\/passwordreq* OR *\/scriptpath* OR *\/times* OR *\/workstations*)))))


...and building on our conversation about `elasticsearch`'s domain-specific languages, here's the JSON DSL version:

In [24]:
!python ../tools/sigmac --target es-dsl --config winlogbeat ../rules/windows/process_creation/win_local_system_owner_account_discovery.yml

{
  "query": {
    "constant_score": {
      "filter": {
        "bool": {
          "should": [
            {
              "bool": {
                "must": [
                  {
                    "bool": {
                      "should": [
                        {
                          "wildcard": {
                            "winlog.event_data.Image.keyword": "*\\\\whoami.exe"
                          }
                        },
                        {
                          "bool": {
                            "must": [
                              {
                                "wildcard": {
                                  "winlog.event_data.Image.keyword": "*\\\\wmic.exe"
                                }
                              },
                              {
                                "wildcard": {
                                  "winlog.event_data.CommandLine.keyword": "*useraccount*"
                                }
                    

Or SQL!

In [26]:
!python ../tools/sigmac --target sql --config windows-audit ../rules/windows/process_creation/win_local_system_owner_account_discovery.yml

(EventID = "4688" AND (((NewProcessName LIKE "%\whoami.exe" OR (NewProcessName LIKE "%\wmic.exe" AND CommandLine LIKE "%useraccount%" AND CommandLine LIKE "%get%") OR (NewProcessName LIKE "%\quser.exe" OR NewProcessName LIKE "%\qwinsta.exe") OR (NewProcessName LIKE "%\cmdkey.exe" AND CommandLine LIKE "%/list%") OR (NewProcessName LIKE "%\cmd.exe" AND CommandLine LIKE "%/c%" AND CommandLine LIKE "%dir %" AND CommandLine LIKE "%\Users\%")) AND NOT ((CommandLine LIKE "% rmdir %"))) OR (((NewProcessName LIKE "%\net.exe" OR NewProcessName LIKE "%\net1.exe") AND CommandLine LIKE "%user%") AND NOT ((CommandLine LIKE "%/domain%" OR CommandLine LIKE "%/add%" OR CommandLine LIKE "%/delete%" OR CommandLine LIKE "%/active%" OR CommandLine LIKE "%/expires%" OR CommandLine LIKE "%/passwordreq%" OR CommandLine LIKE "%/scriptpath%" OR CommandLine LIKE "%/times%" OR CommandLine LIKE "%/workstations%")))))


Or ingesting `sysmon` data in Azure Sentinel:

In [23]:
!python ../tools/sigmac --target ala --config sysmon ../rules/windows/process_creation/win_local_system_owner_account_discovery.yml

SysmonEvent | where EventID == "1" | where (EventID == "1" and (((Image endswith @"\\whoami.exe" or (Image endswith @"\\wmic.exe" and CommandLine contains "useraccount" and CommandLine contains "get") or (Image endswith @"\\quser.exe" or Image endswith @"\\qwinsta.exe") or (Image endswith @"\\cmdkey.exe" and CommandLine contains "/list") or (Image endswith @"\\cmd.exe" and CommandLine contains "/c" and CommandLine contains "dir " and CommandLine contains @"\\Users\\")) and not ((CommandLine contains " rmdir "))) or (((Image endswith @"\\net.exe" or Image endswith @"\\net1.exe") and CommandLine contains "user") and not ((CommandLine contains "/domain" or CommandLine contains "/add" or CommandLine contains "/delete" or CommandLine contains "/active" or CommandLine contains "/expires" or CommandLine contains "/passwordreq" or CommandLine contains "/scriptpath" or CommandLine contains "/times" or CommandLine contains "/workstations")))))
